In [17]:
import pandas as pd
import numpy as np

In [18]:
dataset = pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True)
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],
              'max_features': ['sqrt','log2'],              
            }
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\huawei\anaconda3\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2']},
             scoring='f1_weighted', verbose=3)

In [22]:
print("The best parameter:",grid.best_params_)

The best parameter: {'criterion': 'gini', 'max_features': 'log2'}


In [23]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.383050,0.011547,0.029908,0.000615,gini,sqrt,"{'criterion': 'gini', 'max_features': 'sqrt'}",1.0,0.963889,0.947015,0.982051,1.0,0.978591,0.020697,3
1,0.347968,0.022544,0.030373,0.000599,gini,log2,"{'criterion': 'gini', 'max_features': 'log2'}",1.0,0.982051,0.947015,0.982051,1.0,0.982223,0.019348,1
2,0.325230,0.003170,0.031008,0.002199,entropy,sqrt,"{'criterion': 'entropy', 'max_features': 'sqrt'}",1.0,0.963889,0.929513,0.982051,1.0,0.975090,0.026428,4
3,0.211951,0.055848,0.021267,0.004861,entropy,log2,"{'criterion': 'entropy', 'max_features': 'log2'}",1.0,0.963889,0.964572,0.982051,1.0,0.982102,0.015998,2


In [24]:
grid_pred=grid.predict(x_test)

In [25]:
from sklearn.metrics import f1_score
f1_wght_average=f1_score(y_test,grid_pred,average='weighted')
print("The F1 score final:",f1_wght_average)

The F1 score final: 0.9916844900066377


In [26]:
from sklearn.metrics  import confusion_matrix
cm=confusion_matrix(y_test,grid_pred)
print(cm)

[[45  0]
 [ 1 74]]


In [27]:
from sklearn.metrics import classification_report
report=classification_report(y_test,grid_pred)
print(report)

              precision    recall  f1-score   support

       False       0.98      1.00      0.99        45
        True       1.00      0.99      0.99        75

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [28]:
from sklearn.metrics import roc_auc_score
scr=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print("The roc_auc_score is:",scr)

The roc_auc_score is: 0.9997037037037038


In [29]:
result=grid.predict([[20.0, 70.0, 0.0, 0.0, 123.0, 44.0, 1.0, 135.0, 3.8, 14.6, 44.0,
       5500.0, 4.8, False, False, False, False, True, True, False, False,
       False, False, False, True, False, False]])
print(result)

[False]


C:\Users\huawei\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
